![CC-BY-SA](https://mirrors.creativecommons.org/presskit/buttons/88x31/svg/by-sa.svg)
This notebook was created by [Bernardo Freitas Paulo da Costa](http://www.im.ufrj.br/bernardofpc),
and is licensed under Creative Commons BY-SA

# Libs

In [ ]:
import SDDP, JuMP, ASDDiP, PyPlot

In [ ]:
import LaTeXStrings: @L_str

## Local utils

In [ ]:
function graph_fcfs(m, t, ts, QTrue; filename=nothing)
    qe = QTrue
    qt = ASDDiP.Qtilde(m,t,1,ts)
    qf = ASDDiP.Qfrak(m,t,1,ts)

    fig, (ax1,ax2) = PyPlot.subplots(ncols=2, figsize=(12,4))
    PyPlot.suptitle("Stage $t")

    ax1[:plot](ts, qe, label=L"$\overline{Q}$: exact")
    ax1[:plot](ts, qt, label=L"$\tilde{Q}$: mean of next stage approximations")
    ax1[:plot](ts, qf, label=L"$\mathfrak{Q}$: current stage approximation")
    ax1[:legend]()
    ax1[:set_title]("Future cost functions")
    ax1[:grid]()

    ax2[:plot](ts, qt - qf, label=L"$\tilde{Q} - \mathfrak{Q}$")
    ax2[:plot](ts, qe - qf, label=L"$\overline{Q} - \mathfrak{Q}$")
    ax2[:plot](ts, qe - qt, label=L"$\overline{Q} - \tilde{Q}$")
    ax2[:set_title]("Differences")
    ax2[:legend]()
    ax2[:grid]()

    if filename != nothing
        PyPlot.savefig(filename)
    end
end

# A toy model for ALD-SDDiP

## Description

$$\begin{array}{rl}
      \min  & \mathbb{E}\left[\sum\limits_t \beta^{t-1}|x_t|\right] \\
\text{s.t.} & \quad x_t = x_{t-1} + c_t + \xi_t \\
            & \quad c_t \in \{\pm 1\}
\end{array} $$

## Future cost functions

The corresponding dynamic programming / recursive equations are:

$$ Q_t(x_{t-1},\xi_t) =
  \begin{array}[t]{rl}
  \min\limits_{x_t} & |x_t| + \beta \cdot \mathbb{E}\left[ Q_{t+1}(x_t,\xi_{t+1}) \right] \\
  \text{s.t.}       & x_t = x_{t-1} + c_t + \xi_t \\
                    & c_t \in \{\pm1\}
  \end{array}
$$

The averages will be denoted (as usual) by $\overline{Q}_t(x_{t-1}) = \mathbb{E}\big[ Q_t(x_{t-1},\xi_t) \big]$.

### Cuts and lower approximations

Any lower bound for $\overline{Q}_{t+1}$ is a _cut_.
The maximum of $k$ such cuts is denoted $\mathfrak{Q}_{t+1}^k$, and is constructed incrementally during the SDDP algorithm.
The dynamic programming where we replace $\overline{Q}_{t+t}$ with $\mathfrak{Q}_{t+1}$ yields the backwards functions:
$$ \tilde{Q}_t^k(x_{t-1},\xi_t) =
  \begin{array}[t]{rl}
  \min\limits_{x_t} & |x_t| + \beta \cdot \mathfrak{Q}_{t+1}^k(x_t) \\
  \text{s.t.}       & x_t = x_{t-1} + c_t + \xi_t \\
                    & c_t \in \{\pm1\}
  \end{array}
$$


## Analytic solution using symmetry

In [ ]:
cost(x::Float64) = abs(x)
next_step(x::Float64) = (x >=0 ? x-1 : x+1)

In [ ]:
function Q2_bar(x2::T, nmax::Int, noise::AbstractArray{T,1}, beta=0.5::Float64, t=1::Int) where T
    if t >= nmax
        return 0.0
    end

    nsamples = length(noise)
    ans = 0.0
    for xi in noise
        ans += solve_bin_sym(x2+xi, nmax, noise, beta, t+1)
    end
    return ans/nsamples
end
    
function solve_bin_sym(x::T, nmax::Int, noise::AbstractArray{T,1}, beta=0.5::Float64, t=1::Int) where T
    if t > nmax
        return 0.0
    end
    
    x2 = next_step(x)
    return cost(x2) + beta*Q2_bar(x2, nmax, noise, beta, t)
end

## Data 

### The discount factor $\beta$

We take a relatively "small" decay, $\beta = 0.9$.

In [ ]:
discount = 0.9

### The noise $\xi_t$

Is identically distributed ("periodic system"), and **symmetric** (important for the analytic solution above).

In [ ]:
srand(11111)
A_noise   = 0.4
num_noise = 5
noise = randn(num_noise)
noise = A_noise * [noise; -noise];

## Calculations for a 61-point discretization

In [ ]:
ts = -3:0.1:3

In [ ]:
Qt3 = Vector{Float64}[]
for t = 1:7
    v = [Q2_bar(ti, 8, noise, discount, t) for ti in ts]
    push!(Qt3, v)
end

In [ ]:
fig, (ax1,ax2) = PyPlot.subplots(ncols=2, figsize=(10,4))
for t = 1:7
    ax1[:plot](ts, Qt3[t], label="$t")
    ax2[:plot](ts, Qt3[t]*discount^t, label="$t")
end
ax1[:set_title]("Future cost (value at current time)")
ax2[:set_title]("Future cost (in 1st stage values)")
ax2[:legend](title=L"Stage $t$", bbox_to_anchor=(1,0.5), loc="center left");

The future cost "at current time" can also be interpreted as the future cost at the first stage considering a time horizon of $9-t$ stages.
So, for example, the line corresponding to "stage 3" is also the future cost function at the first stage of a 6-stage problem.

The limit of these curves can be taken as one approximation for the "infinite horizon problem".

# Cost-to-go and Average cost-to-go

In [ ]:
fig, (ax1,ax2) = PyPlot.subplots(ncols=2, figsize=(10,4))
for t = 1:7
    ctg = [solve_bin_sym(ti, 8, noise, discount, t) for ti in ts]
    ax1[:plot](ts, ctg*discount^(t-1), label="$t")
    ax2[:plot](ts, Qt3[t]*discount^t, label="$t")
end
ax1[:set_title]("Cost-to-go")
ax2[:set_title](L"Future cost $\overline{Q}_t(x_{t-1})$")
ax1[:set_xlabel](L"$x_{t-1} + \xi_t$")
ax2[:set_xlabel](L"$x_{t-1}$")
ax2[:legend](title=L"Stage $t$", bbox_to_anchor=(1,0.5), loc="center left");

# Using only Strenghtened benders cuts:

This corresponds to $\rho = 0$, for all stages and iterations.

In [ ]:
ramp_mode = :None

In [ ]:
niters = 100
include("control.jl")

In [ ]:
ts = -3:0.02:3
for t = 1:7
    PyPlot.plot(ts, ASDDiP.Qfrak(m,t,1,ts), label="$t")
end
PyPlot.legend(title=L"Stage $t$")
PyPlot.title(L"Future cost functions $\mathfrak{Q}_t$")
PyPlot.grid();

In [ ]:
ts = -3:0.1:3
for t = 1:7
    QTrue = Qt3[t]*discount^t
    graph_fcfs(m,t, ts, QTrue, filename="/tmp/1d_sb_stage$t.pdf")
end

# Using ALD

We estimate $\displaystyle Lip_t = \beta^{t-1} + \beta^t + \ldots + \beta^{8-1} = \beta^{t-1}(1 + \ldots + \beta^{8-t}) = \frac{1 - \beta^{8+1-t}}{1 - \beta}\beta^{t-1}$.

# First strategy: homothetic

At iteration $n$ and stage $t$, we set
$$\rho_t = \min\left(1, \max\left(0, \frac{n-15}{15}\right)\right) \cdot Lip_t. $$

That is:

- in the first 15 stages, $\rho_t = 0$;
- then increase $\rho_t$ until it reaches $Lip_t$ in 15 stages;
- and keep at $Lip_t$ until the end.

In [ ]:
ramp_mode = :simple

In [ ]:
include("control.jl")

### Graph of FCF

In [ ]:
ts = -3:0.02:3
for t = 1:7
    PyPlot.plot(ts, ASDDiP.Qfrak(m,t,1,ts), label="$t")
end
PyPlot.legend(title="Stage")
PyPlot.title("Future cost functions")
PyPlot.grid();

In [ ]:
ts = -3:0.1:3
for t = 1:7
    QTrue = Qt3[t]*discount^t
    graph_fcfs(m,t, ts, QTrue, filename="/tmp/1d_ald_s_stage$t.pdf")
end

# Second strategy: parallel

At iteration $n$ and stage $t$, we set
$$\rho_t = \min\left(Lip_t, \max\left(0, \frac{n-15}{15}\right)\right). $$

That is:

- in the first 15 stages, $\rho_t = 0$;
- then increase with equal steps at all stages, stopping at $Lip_t$; (so that different stages "saturate" at different times)
- and keep at $Lip_t$ until the end.

In [ ]:
ramp_mode = :parallel

In [ ]:
include("control.jl")

### Graph of FCF

In [ ]:
ts = -3:0.02:3
for t = 1:7
    PyPlot.plot(ts, ASDDiP.Qfrak(m,t,1,ts), label="$t")
end
PyPlot.legend(title="Stage")
PyPlot.title("Future cost functions")
PyPlot.grid();

In [ ]:
ts = -3:0.1:3
for t = 1:7
    QTrue = Qt3[t]*discount^t
    graph_fcfs(m,t, ts, QTrue, filename="/tmp/1d_ald_p_stage$t.pdf")
end

# Third strategy: double of parallel

At iteration $n$ and stage $t$, we set
$$\rho_t = \min\left(2 Lip_t, \max\left(0, \frac{n-15}{15}\right)\right). $$

That is:

- in the first 15 stages, $\rho_t = 0$;
- then increase with equal steps at all stages, stopping at $2 Lip_t$; (so that different stages "saturate" at different times)
- and keep at $2 Lip_t$ until the end.

In [ ]:
ramp_mode = :parallel2

In [ ]:
include("control.jl")

### Graph of FCF

In [ ]:
ts = -3:0.02:3
for t = 1:7
    PyPlot.plot(ts, ASDDiP.Qfrak(m,t,1,ts), label="$t")
end
PyPlot.legend(title="Stage")
PyPlot.title("Future cost functions")
PyPlot.grid();

In [ ]:
ts = -3:0.1:3
for t = 1:7
    QTrue = Qt3[t]*discount^t
    graph_fcfs(m,t, ts, QTrue, filename="/tmp/1d_ald_p2_stage$t.pdf")
end